# Accessing and reusing BigBrain data with DataLad and Boutiques

This tutorial will walk you through the following steps:
1. Finding BigBrain datasets in DataLad
2. Installing and downloading BigBrain datasets
3. Processing BigBrain datasets
4. Uploading data derived from BigBrain

This tutorial is available on Binder and can be done entirely online, without the need for any local software installation. Familiarity with Linux command lines is recommended but not required.

Alternately, if you are familiar with the [Docker](http://docker.io) system and want to run the tutorial on your own computer, you can run all the commands of this tutorial in Docker image `glatard/hws`.

The main tools and platforms involved in this tutorial are [DataLad](https://www.datalad.org), [Boutiques](http://boutiques.github.io), and the [Canadian Open Neuroscience Platform](http://portal.conp.ca). Please refer to the documentation of these tools for additional information.

## Software installation

The following script installs the required software. You may have to adapt them if you're running this notebook outside of Google Colab:

In [ ]:
!git clone https://github.com/glatard/hws.git && (cd hws && ./install.sh)

## Finding BigBrain datasets in DataLad

One of our goals in [HIBALL](https://bigbrainproject.org/hiball.html) is to distribute BigBrain datasets through the uniform interface provided by DataLad. In this part of the tutorial, we will demonstrate how BigBrain data can be downloaded and manipulated using DataLad. A complete introduction to DataLad, including detailed tutorials, is available in the [DataLad handbook](http://handbook.datalad.org/en/latest/index.html).

BigBrain DataLad datasets are available through the web portal of the Canadian Open Neuroscience Platform, available at http://portal.conp.ca. They can be found by entering "BigBrain" in the search field:

![screenshot](figures/search_data.png)

https://portal.conp.ca/search?search=bigbrain&sortKey=conpStatus&sortComparitor=asc&page=1&max_per_page=10&cursor=0&limit=10

## Installing and downloading BigBrain datasets

Once a dataset is identified, instructions on how to download it using DataLad are available in the corresponding dataset page in the CONP portal:

![screenshot](figures/download_instructions.png)

The next steps will go through these instructions.

### Dataset installation

First, the CONP dataset should be installed to your local machine using `datalad install`:

In [ ]:
!datalad install https://github.com/CONP-PCNO/conp-dataset.git
%cd conp-dataset

The CONP DataLad dataset contains many datasets, located under `projects`:

In [ ]:
!ls projects

The specific BigBrain dataset of interest in this tutorial can be installed as follows:

In [ ]:
!datalad install projects/BigBrain

Importantly, this step does not download the data. Instead, it installs a set of links that could be downloaded at a later stage. Feel free to install any other dataset you might be interested in, this won't involve long transfer times!

This dataset contains the 40$\mu$m BigBrain blocks, in the Nifti and MINC formats:

In [ ]:
!ls projects/BigBrain/3D_Blocks/40um/*

It also contains reconstructed 3D volumes at various resolutions and in various spaces:

In [ ]:
!ls projects/BigBrain/3D_Volumes/*/*

### Data download

The actual data can be downloaded on demand, using `datalad get`:

In [ ]:
!datalad get projects/BigBrain/3D_Volumes/MNI-ICBM152_Space/nii/full8_400um_2009b_sym.nii.gz

The data is now available:

In [ ]:
%matplotlib inline
import nibabel as nib
import nilearn.plotting as nilp
im1 = nib.load('projects/BigBrain/3D_Volumes/MNI-ICBM152_Space/nii/full8_400um_2009b_sym.nii.gz')
nilp.view_img(im1.slicer[100:200,100:200,100:200], bg_img=None, cmap='gray', resampling_interpolation='nearest')

# Local data processing with Boutiques
<img src="https://camo.githubusercontent.com/33f89fe4aa4ebb45bf7b404395d29f281700ffb622b0dad304714f29d45ce678/687474703a2f2f626f757469717565732e6769746875622e696f2f696d616765732f6c6f676f2e706e67" alt="Boutiques logo" title="Boutiques logo" />

## What is Boutiques ?

[Intro slides](https://docs.google.com/presentation/d/1w9SC6IMxhTneR1Mac3-RoF8_ps84XDMIexiwpU_0ERo/edit#slide=id.gb8e9b9a204_0_56)

## What makes a tool a Boutiques tool?

A Boutiques tool is any command line tool that is described within a descriptor file following the Boutiques json schema.

In order to execute this tool, input parameters must provided within a json file known as an invocation

## Steps to process data with Boutiques

1. `bosh search` your desired tool
2. use `bosh example` as a guide for creating a valid invocation for the tool
3. Launch the tool with the command `bosh exec launch`

## BoSh search

To facilitate search of the available tools published to Zenodo, a search functionality is built into the the **Bo**utiques**Sh**ell (bosh) command line interface.

Let's take a look at what are the top 10 most pulled descriptors:

In [ ]:
!bosh search

If we have a tool in mind, we can specify the name of the tool within our query to return descriptors with a matching name

In [ ]:
!bosh search fsl

The `--exact` flag can be used to return descriptor names with the exact spelling

In [ ]:
!bosh search fsl --exact

## BoSh example

The `bosh example` command can provide an example combination tool paramaters.

Let's get an example invocation of fslstats (zenodo.4472771)

In [ ]:
!bosh example zenodo.4472771

To get additional optional parameter, the `--complete` flag can be applied

In [ ]:
!bosh example zenodo.4472771 --complete

Today we'll use fslstats to calculate the histogram of one of the bigbrain blocks. We will do that in a new DataLad dataset:

In [ ]:
%cd /content
!datalad create histogram
%cd histogram

In [ ]:
!echo '{"input_file": "../conp-dataset/projects/BigBrain/3D_Volumes/MNI-ICBM152_Space/nii/full8_400um_2009b_sym.nii.gz", "h": 10}' > invocation.json

## Launching tools with `bosh exec launch`

Once we know which Boutiques tool we'd like to use and have created a valid invocation, we are ready to launch our tool.
This can be achieved using the `bosh exec launch` command.

In [ ]:
!bosh exec launch zenodo.4472771 invocation.json

In [ ]:
!mv ../conp-dataset/projects/BigBrain/3D_Volumes/MNI-ICBM152_Space/nii/full8_400um_2009b_sym.txt .
!head full8_400um_2009b_sym.txt

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
hist_data = np.genfromtxt('full8_400um_2009b_sym.txt')
plt.bar(np.arange(len(hist_data[:-1])), hist_data[:-1])
plt.show()

## Other useful features

- Python API
- Can interface with Pydra
- ...

# Adding derived data to DataLad

In [ ]:
!datalad save

In [ ]:
!OSF_TOKEN=<ask_your_instructor> datalad create-sibling-osf --title 'BigBrain histogram' \
  --mode exportonly \
  -s osf-export \
  --description "This carefully acquired data will bring science forward" \
  --public

In [ ]:
!OSF_TOKEN=<ask_your_instructor> git-annex export HEAD --to osf-export-storage

# Use CBRAIN to access and launch tool on BigBrain dataset.

In CBRAIN you will be able to access the BigBrain dataset and to launch tool on it (for example fslstats). 

Following the tutorial [here](https://docs.google.com/presentation/d/1YXFlPkxiGzUyHpbSzpS8X5RviVaosW0vDI0aZ3UlhIE/edit?usp=sharing) we'll launch fslstats on BigBrain dataset. 

You can login to CBRAIN [here](https://portal.cbrain.mcgill.ca/session/new).

# Use SFTP to upload files.

In order to get your files back we will use CBRAIN SFTP.
Documenation can be found [here](https://portal.cbrain.mcgill.ca/doc/manual/uploading.html).




In [ ]:
%mkdir cbrain_outputs
%cd cbrain_outputs

In order to get your file back we will use CBRAIN SFTP. Documenation can be found here.

You will have to type:

- `Your password`
- `get block*`
- `exit`


In [ ]:
!ptyrun sftp -o StrictHostKeyChecking=no -P 7500 YOUR_LOGIN@ace-cbrain-1.cbrain.mcgill.ca

In [ ]:
%ls